In [1]:
data_var = '2023-12-19'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8202,2023-12-19,Brasil Nbb,20:00,Caxias do Sul,Corinthians Paulista,2.48,1.50,152.5,1.87,1.79,2.5,2.09,1.66,6m0iu61j,0.403226,0.666667,0.534759,0.558659,0.069892,0.8,0.4,NaN,NaN,282.422,84.762652,0.300128,150.730,89.130389,0.591325,169.324,202.926,196.30,187.50,0.0,0.0,0.0,0.0,0.348223,0.030912,0.162163,-3.59,-0.718,-2.061281,0.000000,0.0,0.000000,-3.64,-0.728,-0.686813,0.000000,0.0,0.000000
8203,2023-12-19,Eua Nba,00:00,Sacramento Kings,Washington Wizards,1.11,8.65,249.5,1.94,2.00,-15.5,1.99,1.78,IF7j5s29,0.900901,0.115607,0.515464,0.500000,0.016508,0.8,0.6,NaN,NaN,205.638,33.921993,0.164960,573.732,171.727114,0.299316,209.308,649.076,155.00,798.12,0.0,1.0,0.0,0.0,1.092538,0.021536,0.078776,1.44,0.288,0.381944,0.588162,0.7,0.111838,-5.00,-1.000,-7.650000,0.230778,0.1,-0.130778
8204,2023-12-19,Eua Nba,00:30,Los Angeles Lakers,New York Knicks,1.54,2.73,233.5,1.86,2.07,-5.5,2.06,1.87,d66f41HF,0.649351,0.366300,0.537634,0.483092,0.015651,0.2,1.0,NaN,NaN,215.368,71.525855,0.332110,312.576,114.299850,0.365671,188.252,319.614,165.85,370.88,0.0,0.0,0.0,0.0,0.394125,0.075569,0.068372,-0.63,-0.126,-4.285714,0.648170,0.7,0.051830,-0.03,-0.006,-288.333333,0.470176,0.5,0.029824
8205,2023-12-19,Eua Nba,21:30,New Orleans Pelicans,Memphis Grizzlies,1.36,3.55,231.5,1.82,1.99,-7.5,1.99,1.82,Es1a3LWL,0.735294,0.281690,0.549451,0.502513,0.016984,0.4,0.6,NaN,NaN,191.114,19.158027,0.100244,319.654,76.373234,0.238925,219.390,405.750,209.33,432.62,0.0,0.0,0.0,0.0,0.630780,0.063101,0.063101,3.96,0.792,0.454545,0.508985,0.7,0.191015,2.04,0.408,6.250000,0.355806,0.5,0.144194
8206,2023-12-19,Eua Nba,22:00,Milwaukee Bucks,San Antonio Spurs,1.06,14.80,247.5,1.81,2.00,-17.5,2.05,1.77,hYa32unS,0.943396,0.067568,0.552486,0.500000,0.010964,1.0,0.2,NaN,NaN,175.668,21.064518,0.119911,342.070,65.188521,0.190571,175.668,647.848,179.20,322.26,0.0,0.0,0.0,0.0,1.225176,0.070525,0.103660,1.49,0.298,0.201342,0.795291,1.0,0.204709,-5.00,-1.000,-13.800000,0.218181,0.1,-0.118181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8291,2023-12-19,Países Baixos Copa Dbl,16:00,LWD Basket,Feyenoord,1.58,2.29,156.5,1.80,1.86,-3.5,1.98,1.74,Ugg6T2CR,0.632911,0.436681,0.555556,0.537634,0.069593,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.259455,0.023184,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8292,2023-12-19,Rússia Liga Vtb United,09:30,Enisey,Saratov,1.28,3.75,164.5,1.85,1.93,-9.5,1.99,1.78,8lS49egl,0.781250,0.266667,0.540541,0.518135,0.047917,0.0,0.2,NaN,NaN,191.448,80.833803,0.422223,289.506,227.727729,0.786608,0.000,354.246,274.50,693.00,0.0,1.0,0.0,0.0,0.694455,0.029930,0.078776,0.00,0.000,inf,0.000000,0.0,0.000000,2.16,0.432,6.365741,0.000000,0.0,0.000000
8293,2023-12-19,Suécia Basketligan,15:04,Jamtland,Norrkoping,2.01,1.73,161.5,1.84,1.86,-8.5,2.09,1.66,bPwy818e,0.497512,0.578035,0.543478,0.537634,0.075547,0.6,0.4,NaN,NaN,156.710,77.552749,0.494881,117.884,26.615184,0.225774,100.906,123.792,94.01,107.91,0.0,0.0,0.0,0.0,0.105877,0.007644,0.162163,-1.34,-0.268,-3.768657,0.000000,0.0,0.000000,1.84,0.368,1.983696,0.000000,0.0,0.000000
8294,2023-12-19,Ucrânia Superliga Da Fbu,10:00,BIPA,Cherkaski Mavpy,3.58,1.27,145.5,1.87,1.87,7.5,1.92,1.83,jqlSmDlr,0.279330,0.787402,

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,00:30,Eua Nba,Los Angeles Lakers,New York Knicks,233.5,1.86,1.0,Over
